In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-obp

*Tantya sa paggamit: 16 minuto sa isang Eagle r3 processor (PAALALA: Ito ay tantya lamang. Maaaring mag-iba ang iyong runtime.)*

In [ ]:
# This cell is hidden from users;
# it disables linting rules.
# ruff: noqa

## Background

Ang operator backpropagation ay isang teknikang nagsasangkot ng pag-absorb ng mga operasyon mula sa dulo ng isang quantum circuit tungo sa sinusukat na observable, na karaniwang nagpapababa ng lalim ng circuit sa halaga ng karagdagang mga term sa observable. Ang layunin ay mag-backpropagate ng kasing dami ng circuit hangga't maaari nang hindi pinapayagang lumaki nang labis ang observable. Ang isang Qiskit-based na implementasyon ay available sa OBP Qiskit addon, at mas maraming detalye ay matatagpuan sa kaukulang [docs](/guides/qiskit-addons-obp) kasama ang isang [simpleng halimbawa](/guides/qiskit-addons-obp-get-started) para makapagsimula.

Isaalang-alang ang isang halimbawang circuit kung saan ang isang observable $O = \sum_P c_P P$ ay susukatiin, kung saan ang $P$ ay mga Pauli at ang $c_P$ ay mga coefficient. Itakda natin ang circuit bilang isang unitary $U$ na maaaring lohikal na hatiin sa $U = U_C U_Q$ tulad ng ipinapakita sa figure sa ibaba.

![Circuit diagram showing Uq followed by Uc](../docs/images/tutorials/improving-estimation-of-expectation-values-with-operator-backpropagation/logical-partitioning.avif)

Ang operator backpropagation ay nag-aabsorb ng unitary $U_C$ sa observable sa pamamagitan ng pag-evolve nito bilang $O' = U_C^{\dagger}OU_C = \sum_P c_P U_C^{\dagger}PU_C$. Sa madaling salita, ang bahagi ng computation ay ginagawa nang classical sa pamamagitan ng evolusyon ng observable mula sa $O$ tungo sa $O'$. Ang orihinal na problema ay maaari ngayong muling iformula bilang pagsukat ng observable $O'$ para sa bagong mas mababang lalim na circuit na ang unitary ay $U_Q$.

Ang unitary $U_C$ ay kinakatawan bilang isang bilang ng mga slice $U_C = U_S U_{S-1}...U_2U_1$. May maraming paraan para tukuyin ang isang slice. Halimbawa, sa halimbawang circuit sa itaas, ang bawat layer ng $R_{zz}$ at bawat layer ng $R_x$ gates ay maaaring ituring bilang indibidwal na slice. Ang backpropagation ay nagsasangkot ng pagkalkula ng $O' = \Pi_{s=1}^S \sum_P c_P U_s^{\dagger} P U_s$ nang classical. Ang bawat slice $U_s$ ay maaaring katawanin bilang $U_s = exp(\frac{-i\theta_s P_s}{2})$, kung saan ang $P_s$ ay isang $n$-qubit Pauli at ang $\theta_s$ ay isang scalar. Madaling ma-verify na

$$
U_s^{\dagger} P U_s = P \qquad \text{if} ~[P,P_s] = 0,
$$
$$
U_s^{\dagger} P U_s = \qquad cos(\theta_s)P + i sin(\theta_s)P_sP \qquad \text{if} ~{P,P_s} = 0
$$

Sa halimbawa sa itaas, kung ${P,P_s} = 0$, kailangan nating magsagawa ng dalawang quantum circuit, sa halip na isa, upang kalkulahin ang expectation value. Samakatuwid, ang backpropagation ay maaaring magdagdag ng bilang ng mga term sa observable, na nagreresulta sa mas mataas na bilang ng circuit execution. Isang paraan upang payagan ang mas malalim na backpropagation sa circuit, habang pinipigilan ang operator mula sa paglaki nang labis, ay ang mag-truncate ng mga term na may maliliit na coefficient, sa halip na idagdag ang mga ito sa operator. Halimbawa, sa halimbawa sa itaas, maaari tayong pumili na mag-truncate ng term na nagsasangkot ng $P_sP$ kung sakaling ang $\theta_s$ ay sapat na maliit. Ang pag-truncate ng mga term ay maaaring magresulta sa mas kakaunting quantum circuit na isasagawa, ngunit ang paggawa nito ay nagreresulta ng ilang error sa panghuling pagkalkula ng expectation value na proporsyonal sa magnitude ng mga coefficient ng mga na-truncate na term.

Ang tutorial na ito ay nagsasagawa ng isang [Qiskit pattern](/guides/intro-to-patterns) para sa pagsisimula ng quantum dynamics ng isang Heisenberg spin chain gamit ang <a href="https://github.com/Qiskit/qiskit-addon-obp">qiskit-addon-obp</a>.

## Requirements

Bago simulan ang tutorial na ito, siguraduhing mayroon ka ng mga sumusunod na naka-install:

- Qiskit SDK v1.2 o mas bago (`pip install qiskit`)
- Qiskit Runtime v0.28 o mas bago (`pip install qiskit-ibm-runtime`)
- OBP Qiskit addon (`pip install qiskit-addon-obp`)
- Qiskit addon utils (`pip install qiskit-addon-utils`)

## Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import CouplingMap
from qiskit.synthesis import LieTrotter

from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit_addon_utils.slicing import slice_by_gate_types, combine_slices
from qiskit_addon_obp.utils.simplify import OperatorBudget
from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.truncating import setup_budget

from rustworkx.visualization import graphviz_draw

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions

## Part I: Small-scale Heisenberg spin chain
### Step 1: Map classical inputs to a quantum problem
#### Map the time-evolution of a quantum Heisenberg model to a quantum experiment.
Ang package na [qiskit_addon_utils](https://github.com/Qiskit/qiskit-addon-utils) ay nagbibigay ng ilang muling magagamit na functionality para sa iba't ibang layunin.

Ang module nitong [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) ay nagbibigay ng mga function upang lumikha ng mga Heisenberg-like Hamiltonian sa isang ibinigay na connectivity graph.
Ang graph na ito ay maaaring [rustworkx.PyGraph](https://www.rustworkx.org/apiref/rustworkx.PyGraph.html) o [CouplingMap](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.CouplingMap) na nagpapadalì ng paggamit sa mga Qiskit-centric na workflow.

Sa sumusunod, bubuo tayo ng isang linear chain na `CouplingMap` ng 10 qubit.

In [3]:
num_qubits = 10
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif)

Susunod, bubuo tayo ng isang Pauli operator na nagmomodelo ng Heisenberg XYZ Hamiltonian.

$$
{\hat{\mathcal{H}}_{XYZ} = \sum_{(j,k)\in E} (J_{x} \sigma_j^{x} \sigma_{k}^{x} + J_{y} \sigma_j^{y} \sigma_{k}^{y} + J_{z} \sigma_j^{z} \sigma_{k}^{z}) + \sum_{j\in V} (h_{x} \sigma_j^{x} + h_{y} \sigma_j^{y} + h_{z} \sigma_j^{z})}
$$

Kung saan ang $G(V,E)$ ay ang graph ng ibinigay na coupling map.

In [4]:
# Get a qubit operator describing the Heisenberg XYZ model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII', 'IIIIIIIIIX', 'IIIIIIIIIY', 'IIIIIIIIIZ', 'IIIIIIIIXI', 'IIIIIIIIYI', 'IIIIIIIIZI', 'IIIIIIIXII', 'IIIIIIIYII', 'IIIIIIIZII', 'IIIIIIXIII', 'IIIIIIYIII', 'IIIIIIZIII', 'IIIIIXIIII', 'IIIIIYIIII', 'IIIIIZIIII', 'IIIIXIIIII', 'IIIIYIIIII', 'IIIIZIIIII', 'IIIXIIIIII', 'IIIYIIIIII', 'IIIZIIIIII', 'IIXIIIIIII', 'IIYIIIIIII', 'IIZIIIIIII', 'IXIIIIIIII', 'IYIIIIIIII', 'IZIIIIIIII', 'XIIIIIIIII', 'YIIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.39269908+0.j, 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j,
 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j, 0.78539816+0.j,
 1.57079633+0.j, 0.39269908+0.j, 0.

From the qubit operator, we can generate a quantum circuit which models its time evolution.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a handy function do just that:

In [5]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=2),
)
circuit.draw("mpl", style="iqp", scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif" alt="Output of the previous code cell" />

Mula sa qubit operator, maaari tayong lumikha ng quantum circuit na nagmomodelo ng time evolution nito.
Muli, ang module na [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) ay dumarating sa pagliligtas na may kapaki-pakinabang na function upang gawin iyon:

In [6]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 18 slices.


![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif)

### Step 2: Optimize problem for quantum hardware execution
#### Create circuit slices to backpropagate
Tandaan, ang function na ``backpropagate`` ay mag-backpropagate ng buong circuit slice nang sabay-sabay, kaya ang pagpili kung paano maghiwa ay maaaring magkaroon ng epekto sa kung gaano kahusay gumagana ang backpropagation para sa isang partikular na problema. Dito, pagsasamahin natin ang mga gate ng parehong uri sa mga slice gamit ang function na [slice_by_gate_types](https://docs.quantum.ibm.com/api/qiskit-addon-utils/slicing#slice_by_gate_types).

Para sa mas detalyadong talakayan tungkol sa circuit slicing, tingnan ang [how-to guide](https://qiskit.github.io/qiskit-addon-utils/how_tos/create_circuit_slices.html) na ito ng package na [qiskit-addon-utils](https://github.com/Qiskit/qiskit-addon-utils).

In [7]:
op_budget = OperatorBudget(max_qwc_groups=8)

#### Backpropagate slices from the circuit

First we specify the observable to be $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, $N$ being the number of qubits. We will backpropagate slices from the time-evolution circuit until the terms in the observable can no longer be combined into eight or fewer qubit-wise commuting Pauli groups.

In [8]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits=num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j,
 0.1+0.j, 0.1+0.j])

#### Constrain how large the operator may grow during backpropagation
Sa panahon ng backpropagation, ang bilang ng mga term sa operator ay karaniwang lalapit sa $4^N$ nang mabilis, kung saan ang $N$ ay ang bilang ng mga qubit. Kapag ang dalawang term sa operator ay hindi nag-commute nang qubit-wise, kailangan natin ng hiwalay na mga circuit upang makuha ang mga expectation value na tumutugma sa kanila. Halimbawa, kung mayroon tayong 2-qubit observable na $O = 0.1 XX + 0.3 IZ - 0.5 IX$, dahil ang $[XX,IX] = 0$, ang pagsukat sa isang basis lamang ay sapat upang kalkulahin ang mga expectation value para sa dalawang term na ito. Gayunpaman, ang $IZ$ ay anti-commute sa dalawang term. Kaya kailangan natin ng hiwalay na basis measurement upang kalkulahin ang expectation value ng $IZ$. Sa madaling salita, kailangan natin ng dalawang circuit, sa halip na isa, upang kalkulahin ang $\langle O \rangle$. Habang dumarami ang bilang ng mga term sa operator, may posibilidad na dumarami rin ang kinakailangang bilang ng circuit execution.

Ang laki ng operator ay maaaring limitahan sa pamamagitan ng pagtukoy ng ``operator_budget`` kwarg ng function na ``backpropagate``, na tumatanggap ng isang instance ng [OperatorBudget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-simplify#operatorbudget).

Upang kontrolin ang dami ng karagdagang resources (oras) na inilalaan, pinapaghihigpitan natin ang maximum na bilang ng qubit-wise commuting Pauli groups na pinapayagang magkaroon ang backpropagated observable. Dito, tinukoy natin na ang backpropagation ay dapat tumigil kapag ang bilang ng qubit-wise commuting Pauli groups sa operator ay lumampas sa 8.

In [9]:
# Backpropagate slices onto the observable
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
# Recombine the slices remaining after backpropagation
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 6 slices.
New observable has 60 terms, which can be combined into 6 groups.
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif" alt="Output of the previous code cell" />

#### Backpropagate slices from the circuit
Una, tinukoy natin ang observable bilang $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, kung saan ang $N$ ay ang bilang ng mga qubit. Mag-backpropagate tayo ng mga slice mula sa time-evolution circuit hanggang sa ang mga term sa observable ay hindi na maaaring pagsamahin sa walong o mas kaunting qubit-wise commuting Pauli groups.

In [10]:
truncation_error_budget = setup_budget(max_error_per_slice=0.005)

Note that by allocating `5e-3` error per slice for truncation, we are able to remove 1 more slice from the circuit, while remaining within the original budget of eight commuting Pauli groups in the observable. By default, `backpropagate` uses the L1 norm of the truncated coefficients to bound the total error incurred from truncation. For other options refer to the [how-to guide on specifying the p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

In this particular example where we have backpropagated seven slices, the total truncation error should not exceed ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
For further discussion on distributing an error budget across your slices, check out [this how-to guide](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [11]:
# Run the same experiment but truncate observable terms with small coefficients
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", scale=0.6)

Backpropagated 7 slices.
New observable has 82 terms, which can be combined into 8 groups.
After truncation, the error in our observable is bounded by 3.266e-02
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/5e8bae1a-ef18-4eb0-9d2a-1ac7bbdced3b-1.avif" alt="Output of the previous code cell" />

Sa ibaba, makikita mo na nag-backpropagate tayo ng anim na slice, at ang mga term ay pinagsama sa anim at hindi walong grupo. Ito ay nagpapahiwatig na ang pag-backpropagate pa ng isang slice ay magiging sanhi ng paglampas ng bilang ng mga Pauli group sa walong. Maaari nating i-verify na ito ang kaso sa pamamagitan ng pagsusuri sa ibinalik na metadata. Pansinin din na sa bahaging ito, ang circuit transformation ay eksakto. Ibig sabihin, walang mga term ng bagong observable na $O'$ ang na-truncate. Ang backpropagated circuit at ang backpropagated operator ay nagbibigay ng eksaktong kinalabasan tulad ng orihinal na circuit at operator.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

In [13]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif)

Susunod, tutukuyin natin ang parehong problema na may parehong mga hadlang sa laki ng output observable. Gayunpaman, sa pagkakataong ito, maglalaan tayo ng error budget sa bawat slice gamit ang function na [setup_budget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-truncating#setup_budget). Ang mga Pauli term na may maliliit na coefficient ay ita-truncate mula sa bawat slice hanggang mapunan ang error budget, at ang natitirang budget ay idadagdag sa budget ng sumusunod na slice. Tandaan na sa kasong ito, ang transformation dahil sa backpropagation ay approximation dahil ang ilan sa mga term sa operator ay na-truncate.

Upang paganahin ang truncation na ito, kailangan nating i-setup ang ating error budget tulad nito:

In [14]:
pub = (circuit_isa, observable_isa)
bp_pub = (bp_circuit_isa, bp_obs_isa)
bp_trunc_pub = (bp_circuit_trunc_isa, bp_obs_trunc_isa)

Tandaan na sa pamamagitan ng paglalaan ng `5e-3` error sa bawat slice para sa truncation, nakakaya nating mag-alis ng 1 pang slice mula sa circuit, habang nananatili sa loob ng orihinal na budget ng walong commuting Pauli groups sa observable. Bilang default, ang `backpropagate` ay gumagamit ng L1 norm ng mga na-truncate na coefficient upang limitahan ang kabuuang error na natamo mula sa truncation. Para sa iba pang mga opsyon, sumangguni sa [how-to guide on specifying the p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

Sa partikular na halimbawa kung saan nag-backpropagate tayo ng pitong slice, ang kabuuang truncation error ay hindi dapat lumampas sa ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
Para sa karagdagang talakayan tungkol sa pamamahagi ng error budget sa iyong mga slice, tingnan ang [how-to guide na ito](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [15]:
ideal_estimator = Estimator()

# Run the experiments using Estimator primitive to obtain the exact outcome
result_exact = (
    ideal_estimator.run([(circuit, observable)]).result()[0].data.evs.item()
)
print(f"Exact expectation value: {result_exact}")

Exact expectation value: 0.8871244838989416


We shall use <a href="/docs/guides/configure-error-mitigation">resilience_level</a> = 2 for this example.

In [ ]:
options = EstimatorOptions()
options.default_precision = 0.011
options.resilience_level = 2

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run([pub, bp_pub, bp_trunc_pub])

### Step 4: Post-process and return result to desired classical format

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

std_no_bp = job.result()[0].data.stds.item()
std_bp = job.result()[1].data.stds.item()
std_bp_trunc = job.result()[2].data.stds.item()

In [ ]:
print(
    f"Expectation value without backpropagation: {result_no_bp} ± {std_no_bp}"
)
print(f"Backpropagated expectation value: {result_bp} ± {std_bp}")
print(
    f"Backpropagated expectation value with truncation: {result_bp_trunc} ± {std_bp_trunc}"
)

Expectation value without backpropagation: 0.8033194665993642
Backpropagated expectation value: 0.8599808781259016
Backpropagated expectation value with truncation: 0.8868736004169483


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]
stds = [std_no_bp, std_bp, std_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.6, 0.92])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif" alt="Output of the previous code cell" />

## Part B: Scale it up!

Let us now use Operator Backpropagation to study the dynamics of the Hamiltonian of a 50-qubit Heisenberg Spin Chain.

### Step 1: Map classical inputs to a quantum problem

We consider a 50-qubit Hamiltonian $\hat{\mathcal{H}}_{XYZ}$ for the scaled up problem with the same values for the $J$ and $h$ coefficients as in the small-scale example. The observable $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ is also the same as before. This problem is beyond classical brute-force simulation.

In [16]:
num_qubits = 50
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/47cb1ac7-44db-4f96-b49b-e889a920d83c-0.avif" alt="Output of the previous code cell" />

In [17]:
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIII', 'IIIIIIIIIIII

In [18]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIII

### Step 4: Post-process and return result to desired classical format

In [19]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=4),
)
circuit.draw("mpl", style="iqp", fold=-1, scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b10d16cf-95da-42c0-9b47-b2e5a8516c82-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution

In [20]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 36 slices.


We specify the `max_error_per_slice` to be 0.005 as before. However, since the number of slices for this large-scale problem is much higher than the small scale problem, allowing an error of 0.005 per slice may end up creating a large overall backpropagation error. We can bound this by specifying `max_error_total` which bounds the total backpropagation error, and we set its value to 0.03 (which is roughly the same as in the small-scale example).

For this large-scale example, we allow a higher value for the number of commuting groups, and set it to 15.

In [21]:
op_budget = OperatorBudget(max_qwc_groups=15)
truncation_error_budget = setup_budget(
    max_error_total=0.03, max_error_per_slice=0.005
)

Let us first obtain the backpropagated circuit and observable without any truncation.

In [22]:
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 7 slices.
New observable has 634 terms, which can be combined into 12 groups.
Note that backpropagating one more slice would result in 1246 terms across 27 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/164e2f00-25b6-4cf1-98f8-8b2886f007ee-1.avif" alt="Output of the previous code cell" />

Now allowing for truncation, we obtain:

In [23]:
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", fold=-1, scale=0.6)

Backpropagated 10 slices.
New observable has 646 terms, which can be combined into 14 groups.
After truncation, the error in our observable is bounded by 2.998e-02
Note that backpropagating one more slice would result in 1226 terms across 29 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/c05a85bc-e5ca-4e02-8c96-98b28811f335-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif)

## Part B: Scale it up!
Gamitin natin ngayon ang Operator Backpropagation upang pag-aralan ang dynamics ng Hamiltonian ng isang 50-qubit na Heisenberg Spin Chain.

### Step 1: Map classical inputs to a quantum problem
Isinasaalang-alang natin ang isang 50-qubit Hamiltonian na $\hat{\mathcal{H}}_{XYZ}$ para sa scaled up na problema na may parehong mga halaga para sa mga coefficient na $J$ at $h$ tulad ng sa small-scale na halimbawa. Ang observable na $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ ay pareho rin sa dati. Ang problemang ito ay lampas sa classical brute-force simulation.

In [24]:
# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile the backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs_trunc.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

In [25]:
print(
    f"2-qubit depth of original circuit: {circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit: {bp_circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit with truncation: {bp_circuit_trunc_isa.depth(lambda x:x.operation.num_qubits==2)}"
)

2-qubit depth of original circuit: 48
2-qubit depth of backpropagated circuit: 40
2-qubit depth of backpropagated circuit with truncation: 36


### Step 3: Execute using Qiskit primitives

In [26]:
pubs = [
    (circuit_isa, observable_isa),
    (bp_circuit_isa, bp_obs_isa),
    (bp_circuit_trunc_isa, bp_obs_trunc_isa),
]

In [27]:
options = EstimatorOptions()
options.default_precision = 0.01
options.resilience_level = 2
options.resilience.zne.noise_factors = [1, 1.2, 1.4]
options.resilience.zne.extrapolator = ["linear"]

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run(pubs)

Para sa scaled up na problema, isinaalang-alang natin ang oras ng evolusyon bilang $0.2$ na may $4$ na trotter step. Ang problema ay pinili upang ito ay lampas sa classical brute-force simulation, ngunit maaaring i-simulate sa pamamagitan ng tensor network method. Ito ay nagpapahintulot sa atin na i-verify ang kinalalabasan na nakuha sa pamamagitan ng backpropagation sa isang quantum computer gamit ang ideal na kinalabasan.

Ang ideal na expectation value para sa problemang ito, na nakuha sa pamamagitan ng tensor network simulation, ay $\simeq 0.89$.

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

In [ ]:
print(f"Expectation value without backpropagation: {result_no_bp}")
print(f"Backpropagated expectation value: {result_bp}")
print(f"Backpropagated expectation value with truncation: {result_bp_trunc}")

Expectation value without backpropagation: 0.7887194658035515
Backpropagated expectation value: 0.9532818300978584
Backpropagated expectation value with truncation: 0.8913400398926913


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(0.89)
ax.set_ylim([0.6, 0.98])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/047d448f-aebf-45ff-a81b-83b2d5ca866d-1.avif" alt="Output of the previous code cell" />

Tinukoy natin ang `max_error_per_slice` bilang 0.005 tulad ng dati. Gayunpaman, dahil ang bilang ng mga slice para sa large-scale na problema ay mas mataas kaysa sa small scale na problema, ang pagpapahintulot ng 0.005 error sa bawat slice ay maaaring magreresulta ng malaking kabuuang backpropagation error. Maaari nating limitahan ito sa pamamagitan ng pagtukoy ng `max_error_total` na humahadlang sa kabuuang backpropagation error, at itinakda natin ang halaga nito sa 0.03 (na halos pareho sa small-scale na halimbawa).

Para sa large-scale na halimbawang ito, pinapayagan natin ang mas mataas na halaga para sa bilang ng commuting groups, at itinakda natin ito sa 15.